# Bayesian MCMC Analysis of Magnetic Fields in Galaxy Superclusters

This notebook demonstrates a **Bayesian statistical analysis** of magnetic fields in galaxy superclusters.  
The workflow uses **Rotation Measure (RM) Synthesis** from LOFAR observations, with a focus on:

- Selecting real polarized sources from RM catalogs
- Inspecting Faraday depth spectra (FDFs) and Stokes Q/U parameters
- Cleaning catalogs and handling duplicates
- Applying **Bayesian inference with** `emcee` to estimate rotation measures
- Visualizing posterior distributions with `corner`

---

## Why Bayesian?
Astrophysical data often contain significant noise and uncertainty.  
Using Bayesian methods we can:
- Incorporate prior knowledge about expected RM distributions
- Sample full posterior distributions with `emcee`
- Quantify credible intervals and correlations between parameters
- Interpret results visually through posterior plots

This notebook demonstrates not only the astrophysical application,  
but also practical **Bayesian data analysis skills** relevant for data science and research.


### Step 1: Source Selection from RM-Synthesis

##### Note: I selected 3 superclusters based on declination (for LOFAR) and redshift (z<0.1). These 3 were famous ones, but I already also selected MSCC 175 and MSCC 236 (Santiago-Bautista+2020 catalog).


After selecting the LoTSS-DR3 pointings covering the sky area of the supercluster, RM-Synthesis is run.

To select the real sources you work on the RMcatalog.csv. RMsynth_json_all.csv contains some additional rows that are useful to have in the main file. Once you filter with your criteria, you can keep only the useful columns for the real sources and have RMcatalog_real_filtered.csv.

In [ ]:
import pandas as pd
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt

%matplotlib inline

#To make astropy print all rows in a long table
from astropy.table.pprint import conf
conf.max_lines = -1
conf.max_width = -1

In [ ]:
base_directory = './'  
column_to_check = ' real_leak'  

for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    
    if os.path.isdir(folder_path):
        input_file_1 = os.path.join(folder_path, 'RMcatalog.csv')
        input_file_2 = os.path.join(folder_path, 'RMsynth_json_all.csv')
        
        if os.path.exists(input_file_1):
            # Read the input CSV
            catalog1_file = input_file_1
            catalog2_file = input_file_2

            # Read the two catalogs, append rows from catalog2 to catalog1 and fix some formatting
            catalog1 = pd.read_csv(catalog1_file)
            catalog2 = pd.read_csv(catalog2_file)

            column_to_append = catalog2['dFDFth']
            catalog1 = pd.concat([catalog1, column_to_append], axis=1)
            catalog1[' RMspo'] = catalog1[' RMspo'].astype(float)
            catalog1[' poldeg_spo'] = catalog1[' poldeg_spo'].astype(float)

            
            #Check 'real_leak' after merging and filter rows with 'r'
            output_catalog = catalog1[catalog1[' real_leak'] == ' r ']
            output_catalog = output_catalog[(output_catalog[' RMspo'] <= 300) & (-300<=output_catalog[' RMspo'])]
            output_catalog = output_catalog[output_catalog[' num_peaks'] <=4]
           
            #For 463/474 these were specific duplicated fake sources from strong polarized source in the field
            #output_catalog = output_catalog[(output_catalog[' RMspo'] <9) | (output_catalog[' RMspo']>13)]

            #For 278
            #output_catalog = output_catalog[(output_catalog[' RMspo'] <-25) | (output_catalog[' RMspo']>-21)]


            condition = (-5 < output_catalog[' RMspo']) & (output_catalog[' RMspo'] < 5) & (output_catalog[' poldeg_spo'] < 2.0) | (output_catalog[' poldeg_spo'] > 30.0)
            filtered_catalog1 = output_catalog[~condition]


            # Specify the columns to keep in the output catalog
            selected_columns = ['field', ' isl', ' xpix', ' ypix', ' I_144MHz', ' RMspo',
                                ' polint_spo', ' poldeg_spo', ' real_leak',' ra', ' dec',
                                ' glon', ' glat', 'dFDFcorMAD', 'dFDFth', 'phiPeakPIfit_rm2',
                                'dPhiPeakPIfit_rm2', 'ampPeakPIfit', 'dAmpPeakPIfit',
                                'polAngleFit_deg', 'dPolAngleFit_deg']

            # Create the output catalog with selected columns
            filtered_catalog1 = filtered_catalog1[selected_columns]
            filtered_catalog1['SNR_MAD'] = filtered_catalog1['ampPeakPIfit'] / filtered_catalog1['dFDFcorMAD']
            filtered_catalog1['SNR_th'] = filtered_catalog1['ampPeakPIfit'] / filtered_catalog1['dFDFth']

            #More conservative? Keep?
            filtered_catalog1 = filtered_catalog1[filtered_catalog1['SNR_th']>=8]
            # Save the output catalog to a CSV file
            
            output_file = os.path.join(folder_path, 'RMcatalog_real_filtered.csv')
            filtered_catalog1.to_csv(output_file, index=False)            

This next snippet creates ds9 region file for the selected sources, one can edit as desired:

In [ ]:
base_directory = './'  

ra_column = ' ra'  
dec_column = ' dec' 
isl_column = ' isl'

for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    
    if os.path.isdir(folder_path):
        filtered_csv_file = os.path.join(folder_path, 'RMcatalog_real_filtered.csv')
        
        if os.path.exists(filtered_csv_file):
            df = pd.read_csv(filtered_csv_file)

            # Create a DS9 region file
            region_file = os.path.join(folder_path, 'r_regions.reg')
            
            with open(region_file, 'w') as f:
                f.write("# Region file format: DS9 version 4.1\n")
                f.write("global color=red dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
                f.write("fk5\n")
                
                for index, row in df.iterrows():
                    ra = row[ra_column]
                    dec = row[dec_column]
                    isl_n=row[isl_column]
                    f.write(f"circle({ra}, {dec}, 200\") # text={{{isl_n}}}\n")


### Step 2: Faraday Depth Spectra (FDF) Inspection

After the first selection one should check FDFs to make sure the selected source is real. With the next code you visualize each FDF and Q/U from the catalogs created for each pointing. The data are in the folder QUspec.
It can be edited as desired of course.

In [ ]:
base_directory = './'  
%matplotlib inline
def calculate_fdf(q, u):
    return np.sqrt(q**2 + u**2)

isl_column = ' isl'
field_column = 'field'
mad_column = 'dFDFcorMAD'
th_column = 'dFDFth'
snr_column = 'SNR_th'
rm_column = ' RMspo'
pol_column = ' polint_spo'

for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    
    if os.path.isdir(folder_path):
        filtered_csv_file = os.path.join(folder_path, 'RMcatalog_real_filtered.csv')
        print(filtered_csv_file)
        if os.path.exists(filtered_csv_file):
            df = pd.read_csv(filtered_csv_file)

            # 
            for index, row in df.iterrows():
                isl_n=row[isl_column]
                field=row[field_column]
                
                mad=row[mad_column]
                th=row[th_column]
                
                snr=int(row[snr_column])
                rm=row[rm_column]
                polint=row[pol_column]
         
                filename_fdf = os.path.join(folder_path,'QUspec',f'lofar_QU_err_{isl_n}_FDFclean.dat')  
                filename_cl = os.path.join(folder_path,'QUspec',f'lofar_QU_err_{isl_n}_FDFmodel.dat')

                data = np.loadtxt(filename_fdf)
                cleanc=np.loadtxt(filename_cl)
                # Extract columns
                x = data[:, 0]
                q = data[:, 1]
                u = data[:, 2]

                # Extract columns
                x_c = cleanc[:, 0]
                q_c = cleanc[:, 1]
                u_c = cleanc[:, 2]


                # Calculate 
                fdf = calculate_fdf(q, u)
                clc = calculate_fdf(q_c, u_c)
               
                fig, axs = plt.subplots(1, 2, figsize=(10, 4))

                print('Field', field)
                axs[0].plot(x, fdf, linestyle='-', color='dodgerblue', linewidth=0.8)
                #axs[0].plot(x_c, clc, linestyle='-', color='red', label='Clean comp.')
                axs[0].plot(rm, polint, marker='x', color='red')
                axs[0].axvline(rm, 0, color='red', linestyle='--', linewidth=0.6, label='Real peak')
                axs[0].axhline(th, 0, color='navy', linestyle='--', linewidth=1, label=r'$\sigma_{QU}$', zorder=5)
                #axs[0].axhline(mad, 0, color='orange', linestyle='--', linewidth=0.8,label='mad', zorder=5)
                axs[0].set_xlabel(r'$\phi$ $[rad/m^{2}]$ ')
                axs[0].set_ylabel(r'|FDF| [Jy beam$^{-1}$]')
                axs[0].set_title(f'{field} - Plot of FDF {isl_n} - SNR {snr}')
                axs[0].set_xlim(-400, 400)
                axs[0].legend(loc='upper right')
                axs[0].grid(True, color='grey', linestyle='--', linewidth=0.2, alpha=0.5, zorder=0)
                
                #axs[1].plot(x, fdf, linestyle='-', color='b',alpha=0.7, label='PI')
                axs[1].plot(x, q, linestyle='-', color='green', alpha=0.7, label='Re (Q)')
                axs[1].plot(x, u, linestyle='-', color='gold', alpha=0.7, label='Im (U)')
                #axs[1].plot(rm, polint, marker='x', color='green')
                axs[1].axhline(th, 0, color='black', linestyle='--', linewidth=1, label=r'$\sigma_{QU}$', zorder=5)
                axs[1].axhline(-th, 0, color='black', linestyle='--', linewidth=1, zorder=5)
                axs[1].set_ylim(-5*th, 5*th)
                axs[1].set_xlim(-400, 400)
                #axs[1].fill_between(x, mad, -mad, color='grey', label='dFDFcorMAD', alpha=0.7, zorder=15)
                axs[1].set_xlabel(r'$\phi$ $[rad/m^{2}]$ ')
                axs[1].set_ylabel(r'[Jy beam$^{-1}$]')
                #axs[1].set_title(f'{field} - Plot of FDF {isl_n} - SNR {snr}')
                axs[1].legend(loc='upper right')
                
                plt.tight_layout()
                #plt.savefig(f'./field_{field}_isl{isl_n}_snr{snr}.pdf')
                plt.show()
                # Close the current figure
                plt.close()


plt.ioff()

At this point, you may want to go back to remove additional sources from each catalog. When you have a final selection for each pointing, you can merge them to produce a final catalog for each supercluster (completely optional, just what I did to keep things in order):

In [ ]:
import pandas as pd
import glob
import os

folder_pattern = './catalogs/superclusters/278/nonDR2/rmsynth_*'  # Adjust this pattern based on your folder structure

folders = glob.glob(folder_pattern)
#print(folders)

combined_df = pd.DataFrame()

for folder in folders:
    file_pattern = os.path.join(folder, 'RMcatalog_real_filtered.csv')
    file_paths = glob.glob(file_pattern)
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined df to a new CSV file
combined_df.to_csv('./final_real_sources.csv', index=False)

LOFAR pointings overlap. To remove duplicate sources, one keeps the one with higher SNR:

In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob
import os


def centers_duplicates(coord1, coord2, shift=100 * u.arcsecond):
    return coord1.separation(coord2) <= shift

base_directory = './catalogs/superclusters/'  


for folder in os.listdir(base_directory):
    folder_path = os.path.join(base_directory, folder)
    
    if os.path.isdir(folder_path):
        filtered_csv_file = os.path.join(folder_path, 'final_real_sources.csv')
        
        if os.path.exists(filtered_csv_file):
            df = pd.read_csv(filtered_csv_file)

            rows_to_drop = []

            for i, row1 in df.iterrows():
                coord1 = SkyCoord(row1[' ra'], row1[' dec'], unit='deg')

                if i not in rows_to_drop:
                    for j, row2 in df.loc[df.index > i].iterrows():
                        coord2 = SkyCoord(row2[' ra'], row2[' dec'], unit='deg')
                        
                        if centers_duplicates(coord1, coord2):
                            rows_to_drop.append(i) if row1['SNR_th'] <= row2['SNR_th'] else rows_to_drop.append(j)

            df_cleaned = df.drop(index=rows_to_drop)

            # Reset index
            df_cleaned.reset_index(drop=True, inplace=True)
            df_cleaned.to_csv(filtered_csv_file, index=False)


### Galactic RM

At this point, it would be easier to work with fits files. You can easily convert csv to fits with topcat. 

To compute the GRM at the location of the final sources:

First, you'll need to download the relevant galactic map file from here: https://wwwmpa.mpa-garching.mpg.de/~ensslin/research/data/faraday_revisited.html. Then make sure that you are reading the correct columns name (these were mine). [If you use this for NVSS sources, change the parameter nside to 256].

In [ ]:
from astropy.io import fits
import healpy as hp
import numpy as np
#from astropy_healpix import HEALPix
#from astropy_healpix import healpy as hp
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
import h5py
import math as m
from astropy.stats import mad_std

# https://wwwmpa.mpa-garching.mpg.de/~ensslin/research/data/faraday_revisited.html
#file = 'faraday_sky_w_ff.hdf5' #noLOFAR
file = 'faraday2020v2.hdf5' #LOFAR
f = h5py.File(file, 'r')

#fsky = f[u'faraday sky']
#n1 = fsky.get(u'mean')
#n2 = fsky.get(u'std')

n1 = f.get(u'faraday_sky_mean') #for LOFAR
n2 = f.get(u'faraday_sky_std') #for LOFAR


RMmap = np.array(n1)
RMerrmap = np.array(n2)

#catalogpath= '/Users/shane/Documents/LOFAR/GRG_LOTSS/DDFacet_cube/DR2_polcats/polcat_May2020/'
table = 'catalog.fits'
tabledata = fits.open(table)[1].data

ra = tabledata.field('ra')
dec = tabledata.field('dec')
rm_peak = tabledata.field('phiPeakPIfit_rm2')
rm_peak_err = tabledata.field('dPhiPeakPIfit_rm2')

#rm_peak = tabledata.field('rm')
#rm_peak_err = tabledata.field('rm_err')


c_icrs = SkyCoord(ra=ra, dec=dec, frame='icrs', unit='deg')

glon = c_icrs.galactic.l.degree
glat = c_icrs.galactic.b.degree 

#####################
# average over disk #
#####################
GRM = np.zeros(len(ra))
GRMerr = np.zeros(len(ra))
RRM = np.zeros(len(ra))
RRMerr = np.zeros(len(ra))

for i in range(len(ra)):
    theta = glon[i]
    phi = glat[i]
      #GRMval = hp.pixelfunc.get_interp_val(RMmap, theta, phi, lonlat=True)
      #print glon[i], glat[i]
    vec = hp.ang2vec(glon[i], glat[i], lonlat=True)
    ipix_disc = hp.query_disc(nside=512, vec=vec, radius=np.radians(0.5))  #nside=512 lofar, nside=256 not lofar
    GRMval = np.mean(RMmap[ipix_disc])
    GRM[i] = GRMval
      #GRMerrval = hp.pixelfunc.get_interp_val(RMerrmap, theta, phi, lonlat=True)
      #GRMerrval = np.mean(RMerrmap[ipix_disc]) 
    GRMerrval = np.mean(RMerrmap[ipix_disc])/m.sqrt(len(ipix_disc)) #consistent with Carretti
    GRMerr[i] = GRMerrval
    
    RRM[i]=rm_peak[i]-GRMval
    RRMerr[i]=rm_peak_err[i]+GRMerrval

# add columns to fits table
orig_table = fits.open(table)[1].data
orig_cols = orig_table.columns
new_cols = fits.ColDefs([
  #fits.Column(name='l', format='D', array=glon),
  #fits.Column(name='b', format='D', array=glat),
  fits.Column(name='GRM_wff_v2022', format='D', array=GRM),
  fits.Column(name='GRMerr_wff_v2022', format='D', array=GRMerr),
  fits.Column(name='RRM_v2022', format='D', array=RRM),
  fits.Column(name='RRMerr_v2022', format='D', array=RRMerr)])
hdu = fits.BinTableHDU.from_columns(orig_cols + new_cols)
hdu.writeto('./name.fits',overwrite=True)

### LoTSS DR2 and NVSS selection

Now you want sources from other catalogs that cover the extension of the supercluster. I used DR2 and NVSS. From the catalogs, I defined a radius around the center of the supercluster and selected al sources withing that radius. At the end, it creates a catalog and a ds9 region file.

In [ ]:
import os
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table


def process_catalog(ra_c, dec_c, mscc):
    #catalog_path = os.path.join('catalogs/superclusters', 'NVSS.fits')
    catalog_path = os.path.join('catalogs/superclusters', 'LoTSS_DR2_RMGrid_v1_RMTable.fits')
    if os.path.exists(catalog_path):
        catalog = Table.read(catalog_path, format='fits')
        selected_points = []
        
        target_coord = SkyCoord(ra=ra_c*u.deg, dec=dec_c*u.deg, frame='icrs')

        for row in catalog:
            ra_j2000_str = row['ra']
            dec_j2000_str = row['dec']

            # Convert RA and Dec from string to SkyCoord --- this is needed if coords not in deg already
            #catalog_coord = SkyCoord(ra=ra_j2000_str, dec=dec_j2000_str, unit=(u.hourangle, u.deg))
            catalog_coord = SkyCoord(ra=ra_j2000_str*u.deg, dec=dec_j2000_str*u.deg, frame='icrs')

            if target_coord.separation(catalog_coord).degree <= 19:
                selected_points.append((catalog_coord.ra.deg, catalog_coord.dec.deg))
                
                
        create_ds9_region_file(selected_points, ra_c, dec_c, mscc)
        
        selected_catalog = catalog.copy()
        mask = [False] * len(selected_catalog)
        for i, row in enumerate(selected_catalog):
            #catalog_coord = SkyCoord(ra=row['ra'], dec=row['dec'], unit=(u.hourangle, u.deg))
            catalog_coord = SkyCoord(ra=row['ra']*u.deg, dec=row['dec']*u.deg, frame='icrs')

            if target_coord.separation(catalog_coord).degree <= 19:
                mask[i] = True

        selected_catalog = selected_catalog[mask]
        #selected_catalog.write(os.path.join('catalogs/superclusters', f'{mscc}_nvss_rm_cat.fits'), format='fits', overwrite=True)
        selected_catalog.write(os.path.join('catalogs/superclusters', f'{mscc}_lofar_rm_cat.fits'), format='fits', overwrite=True)


        
def create_ds9_region_file(points, ra_c, dec_c, mscc):
    #region_file_path = os.path.join('catalogs/superclusters', f'{mscc}_nvss_rm_cat_points.reg')
    region_file_path = os.path.join('catalogs/superclusters', f'{mscc}_lofar_rm_cat_points.reg')
    
    with open(region_file_path, 'w') as f:
        f.write("# Region file format: DS9 version 4.1\n")
        f.write("global color=red dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        f.write("fk5\n")
        for ra, dec in points:
            f.write(f'point({ra}, {dec}) # point=cross\n')
    print(f"DS9 region file created for {ra_c}, {dec_c}: {region_file_path}")


#This is the starting file containing a list of superclusters with their center coordinates. 
fits_file = Table.read('catalogs/superclusters/mscc_zdec_select_final.fits')

for row in fits_file:
    ra_c = row['RAJ2000']  # Assuming 'ra' is the column name
    dec_c = row['DEJ2000']
    mscc = row ['MSCC'] # Assuming 'dec' is the column name
    process_catalog(ra_c,dec_c, mscc)


#### Note: the removal of duplicates between catalogs can be easily done in Topcat. Which is what I did: you select the two catalogs and the radius inside of which it finds a duplicate (NVSS resolution). Otherwise you can use similar code as before. Between NVSS and LOFAR duplicates I kept LOFAR.

### Density cubes

The catalog of superclusters I used is this one: https://cdsarc.u-strasbg.fr/viz-bin/cat/J/A+A/637/A31,
and the density profile is the universal galaxy cluster density profile from Pratt+2022 (https://www.aanda.org/articles/aa/abs/2022/09/aa43074-22/aa43074-22.html).

To create the density cubes you'll need the catalog with the coordinates, redshift, virial radius of each supercluster member. I run this on lofar6 but should work on normal machine also.

In [ ]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)


def compute_r500(Rv, zred):
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    kpc_per_degree = cosmo.kpc_proper_per_arcmin(zred).to(u.kpc / u.degree).value
    R500 = Rv * (0.1**(1/3))  # deg
    R500_cm = R500 * kpc_per_degree * (3.086e21)  # cm
    return R500, R500_cm

def compute_m500_and_norm(R500_cm, zred):
    msun = 1.989e+33  # g
    omegaM = 0.3
    omegaL = 0.7
    h70 = 0.7
    alpha_r = 2.09
    alpha_m = 0.22
    
    rho_500 = 4.6e-27 * ((omegaM * (1 + zred)**3) + omegaL)
    M500 = ((4 * np.pi / 3) * rho_500 * (R500_cm)**3) / msun
    norm = ((((omegaM * (1 + zred)**3) + omegaL)**(1/2))**alpha_r) * (((M500 * h70) / (5 * 10**14))**alpha_m)
    
    return rho_500, M500, norm

def gnfw_density(xx, norm, rho_500):
    f0 = 1.20
    xs = 0.28
    a = 0.42
    b = 0.78
    g = 1.52
    
    rho_m = (f0 * norm) / (((xx / xs)**a) * (1 + (xx / xs)**g)**((3 * b - a) / g))    
    return rho_m * rho_500

def median_density_3d(x, y, z, x0, y0, z0, R500, R500_cm, norm, rho_500):
    r = np.sqrt((x - x0)**2 + (y - y0)**2 + (z - z0)**2)
    xx = r / R500
    density = np.zeros_like(xx)
    valid_mask = xx <= 10
    density[valid_mask] = gnfw_density(xx[valid_mask], norm, rho_500)
    
    return density

def calculations(clusters, cosmo):
    calcs = []
    for cluster in clusters:
        ra0, dec0, z0, Rv, zred = cluster
        R500, R500_cm = compute_r500(Rv, zred)
        rho_500, M500, norm = compute_m500_and_norm(R500_cm, zred)
        calcs.append((ra0, dec0, z0, R500, R500_cm, norm, rho_500))
    return calcs

def create_density_cube(ra_range, dec_range, z_range, clusters, pixel_scale, z_resolution):
    ra_min, ra_max = ra_range
    dec_min, dec_max = dec_range
    z_min, z_max = z_range
    
    ra_size = int((ra_max - ra_min) / pixel_scale)
    dec_size = int((dec_max - dec_min) / pixel_scale)
    z_size = z_resolution
    print(ra_size, dec_size)
    
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    
    ra_grid = np.linspace(ra_max, ra_min, ra_size)
    dec_grid = np.linspace(dec_min, dec_max, dec_size)
    z_grid = np.linspace(z_min, z_max, z_size)
    
    lum_distances = cosmo.luminosity_distance(z_grid).to(u.Mpc).value
    lum_distances_deg = (lum_distances * 1e3) / cosmo.kpc_proper_per_arcmin(z_grid).to(u.kpc / u.degree).value
    
    density_cube = np.zeros((ra_size, dec_size, z_size))
    
    precomputed_clusters = calculations(clusters, cosmo)
    
    # Create meshgrid for all grid points
    ra_grid_mesh, dec_grid_mesh, z_grid_mesh = np.meshgrid(ra_grid, dec_grid, lum_distances_deg, indexing='ij')
    
    for vals in precomputed_clusters:
        ra0, dec0, z0, R500, R500_cm, norm, rho_500 = vals
        ra_pix = int((ra0 - ra_min) / pixel_scale)
        dec_pix = int((dec0 - dec_min) / pixel_scale)
        z_pix = int((z0 - z_min) * z_size / (z_max - z_min))
        
        density_values = median_density_3d(
            ra_grid_mesh, dec_grid_mesh, z_grid_mesh,
            ra0, dec0, lum_distances_deg[z_pix], R500, R500_cm, norm, rho_500
        )
        
        density_cube += density_values
    
    return density_cube

def adjust_wcs(fits_file, ra_size, dec_size, z_size, pixel_scale, z_resolution, ra_range, dec_range, z_range):
    ra_min, ra_max = ra_range
    dec_min, dec_max = dec_range
    z_min, z_max = z_range

    with fits.open(fits_file) as hdul:
        w = WCS(hdul[0].header)
        w = WCS(naxis=3)
    
    w.wcs.crpix = [ra_size / 2, dec_size / 2, z_size / 2]
    w.wcs.cdelt = [pixel_scale, pixel_scale, (z_max - z_min) / z_size]
    w.wcs.crval = [ra_min + (ra_max - ra_min) / 2, dec_min + (dec_max - dec_min) / 2, (z_min + z_max) / 2]
    w.wcs.ctype = ["RA---TAN", "DEC--TAN", "VELO-LSR"]
    
    return w

def save_to_fits(data, filename):
    hdu = fits.PrimaryHDU(data)
    #hdu.header.update(wcs.to_header())
    hdul = fits.HDUList([hdu])
    hdul.writeto(filename, overwrite=True)

def read_clusters_from_fits(filename):
    with fits.open(filename) as hdul:
        data = hdul[1].data
        clusters = [(row['RAJ2000'], row['DEJ2000'], row['z'], row['Rvir'], row['z']) for row in data]
    return clusters




######################## These were the superclusters I analyzed  #############################

#474 
ra_range = [221, 257]  # RA range in degrees
dec_range = [0, 34]  # Dec range in degrees
z_range = [0.025, 0.05]  # Redshift range
pixel_scale = 100 / 3600  # Pixel scale in degrees
z_resolution = 100  # Number of grid points along the redshift dimension

# Read clusters from FITS file
cluster_fits_file = 'catalogs/superclusters/474/mscc_474_cat.fits'
clusters = read_clusters_from_fits(cluster_fits_file)

# Create density cube
density_cube = create_density_cube(ra_range, dec_range, z_range, clusters, pixel_scale, z_resolution)
save_to_fits(density_cube, 'catalogs/superclusters/474/density_cube_mesh.fits')



#278
ra_range = [148, 189]  
dec_range = [6, 46]  
z_range = [0.02, 0.045]  
pixel_scale = 100 / 3600 
z_resolution = 100 

# Read clusters from FITS file
cluster_fits_file = 'catalogs/superclusters/278/mscc_278_cat.fits'
clusters = read_clusters_from_fits(cluster_fits_file)

# Create density cube
density_cube = create_density_cube(ra_range, dec_range, z_range, clusters, pixel_scale, z_resolution)
save_to_fits(density_cube, 'catalogs/superclusters/278/density_cube_mesh.fits')


#463
ra_range = [215, 248] 
dec_range = [12, 44]  
z_range = [0.05, 0.095] 
pixel_scale = 100 / 3600 
z_resolution = 100  

# Read clusters from FITS file
cluster_fits_file = 'catalogs/superclusters/463/mscc_463_cat.fits'
clusters = read_clusters_from_fits(cluster_fits_file)

# Create density cube
density_cube = create_density_cube(ra_range, dec_range, z_range, clusters, pixel_scale, z_resolution)
save_to_fits(density_cube, 'catalogs/superclusters/463/density_cube_mesh.fits')

This is to attach a WCS to the cube. 

In [ ]:
from astropy.wcs import WCS
from astropy.io import fits

def create_wcs(ra_range, dec_range, z_range, pixel_scale, z_resolution):
    ra_min, ra_max = ra_range
    dec_min, dec_max = dec_range
    z_min, z_max = z_range
    
    # Define the WCS object
    wcs = WCS(naxis=3)
    
    # Axis 1: RA
    wcs.wcs.crpix[0] = (ra_max - ra_min) / (2 * pixel_scale)
    wcs.wcs.cdelt[0] = pixel_scale  # pixel size in degrees
    wcs.wcs.crval[0] = (ra_max + ra_min) / 2
    wcs.wcs.ctype[0] = 'RA---SIN'
    wcs.wcs.cunit[0] = 'deg'
    
    # Axis 2: Dec
    wcs.wcs.crpix[1] = (dec_max - dec_min) / (2 * pixel_scale)
    wcs.wcs.cdelt[1] = pixel_scale # pixel size in degrees
    wcs.wcs.crval[1] = (dec_max + dec_min) / 2
    wcs.wcs.ctype[1] = 'DEC--SIN'
    wcs.wcs.cunit[1] = 'deg'
    
    # Axis 3: Redshift
    wcs.wcs.crpix[2] = z_resolution / 2
    wcs.wcs.cdelt[2] = (z_max - z_min) / z_resolution
    wcs.wcs.crval[2] = (z_max + z_min) / 2
    wcs.wcs.ctype[2] = 'REDSHIFT'
    wcs.wcs.cunit[2] = ''
    
    # Additional keywords
    wcs.wcs.radesys = 'ICRS'
    wcs.wcs.lonpole = 180.0
    wcs.wcs.latpole = 17.449952777778
    
    return wcs

def save_density_cube_with_wcs(input_fits, output_fits, ra_range, dec_range, z_range, pixel_scale, z_resolution):
    # Read the existing density cube from FITS file
    ra_min, ra_max = ra_range
    dec_min, dec_max = dec_range
    z_min, z_max = z_range

    with fits.open(input_fits) as hdul:
        density_cube = hdul[0].data
    
    # Create WCS object
    wcs = create_wcs(ra_range, dec_range, z_range, pixel_scale, z_resolution)
    
    # Create the FITS header
    header = wcs.to_header()

    header['CRPIX1'] = wcs.wcs.crpix[0]
    header['CRPIX2'] = wcs.wcs.crpix[1]
    header['CRPIX3'] = wcs.wcs.crpix[2]
    
    header['CDELT1'] = -wcs.wcs.cdelt[0]
    header['CDELT2'] = wcs.wcs.cdelt[1]
    header['CDELT3'] = wcs.wcs.cdelt[2]
    
    header['CRVAL1'] = wcs.wcs.crval[0]
    header['CRVAL2'] = wcs.wcs.crval[1]
    header['CRVAL3'] = wcs.wcs.crval[2]
    
    header['CTYPE1'] = wcs.wcs.ctype[0]
    header['CTYPE2'] = wcs.wcs.ctype[1]
    header['CTYPE3'] = wcs.wcs.ctype[2]
    
    header['CUNIT1'] = str(wcs.wcs.cunit[0])
    header['CUNIT2'] = str(wcs.wcs.cunit[1])
    header['CUNIT3'] = str(wcs.wcs.cunit[2])
    
    header['RADESYS'] = wcs.wcs.radesys
    header['LONPOLE'] = wcs.wcs.lonpole
    header['LATPOLE'] = wcs.wcs.latpole
    
    # Create the Primary HDU with the density cube data and WCS header
    hdu = fits.PrimaryHDU(density_cube, header=header)
    
    # Write to a new FITS file
    hdul_new = fits.HDUList([hdu])
    hdul_new.writeto(output_fits, overwrite=True)

#474
#ra_range = [221, 257]  # RA range in degrees
#dec_range = [0, 34]  # Dec range in degrees
#z_range = [0.025, 0.05]  # Redshift range
#pixel_scale = 100 / 3600  # Pixel scale in degrees
#z_resolution = 100  # Number of grid points along the redshift dimension
#save_density_cube_with_wcs('catalogs/superclusters/474/density_cube_mesh_474.fits', 'catalogs/superclusters/474/density_cube_mesh_474_wcs.fits', ra_range, dec_range, z_range, pixel_scale, z_resolution)

#278

ra_range = [148, 189]  # RA range in degrees
dec_range = [6, 46]  # Dec range in degrees
z_range = [0.02, 0.045]  # Redshift range
pixel_scale = 100 / 3600  # Pixel scale in degrees
z_resolution = 100  # Number of grid points along the redshift dimension
save_density_cube_with_wcs('catalogs/superclusters/278/density_cube_mesh_278.fits', 'catalogs/superclusters/278/3D/density_cube_mesh_278_wcs.fits', ra_range, dec_range, z_range, pixel_scale, z_resolution)

#463
#ra_range = [215, 248]  # RA range in degrees
#dec_range = [12, 44]  # Dec range in degrees
#z_range = [0.05, 0.095]  # Redshift range
#pixel_scale = 100 / 3600  # Pixel scale in degrees
#z_resolution = 100  # Number of grid points along the redshift dimension
#save_density_cube_with_wcs('catalogs/superclusters/463/density_cube_mesh_463.fits', 'catalogs/superclusters/463/3D/density_cube_mesh_463_wcs.fits', ra_range, dec_range, z_range, pixel_scale, z_resolution)


At this point we can compute the mean density crossed by the polarized emission of each catalogued source. 

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np

def mean_density_per_source(density_cube_fits, catalog_fits, output_catalog_fits):
    with fits.open(density_cube_fits) as hdul:
        density_cube = hdul[0].data
        print(density_cube.shape[0],density_cube.shape[1], density_cube.shape[2])
        wcs = WCS(hdul[0].header)
        print(wcs)
        
    # Read the catalog of sources
    with fits.open(catalog_fits) as hdul:
        catalog_data = hdul[1].data
        catalog_header = hdul[1].header
    
    mean_densities = np.zeros(len(catalog_data))

    for i, source in enumerate(catalog_data):
        ra = source['ra']
        dec = source['dec']
        #print(dec)
        
        # Convert RA and Dec to pixel coordinates
        #print(ra,dec)
        pixel_coords = wcs.world_to_pixel_values(ra, dec, 1)
        x_pix, y_pix = pixel_coords[0], pixel_coords[1]
        #print(x_pix, y_pix)
      
    # Check if the coordinates are within bounds and are not NaN
        if not np.isnan(x_pix) and not np.isnan(y_pix):
            x_pix, y_pix = int(x_pix), int(y_pix)
            if 0 <= x_pix < density_cube.shape[0] and 0 <= y_pix < density_cube.shape[1]:
                # Extract the density values along the redshift axis
                density_values = density_cube[x_pix, y_pix, :]
                
                # Compute the mean density over the redshift axis
                mean_density = np.mean(density_values)
                

                mean_densities[i] = mean_density
                
                
    # Add the mean density values as a new column in the catalog
    col_mean_density = fits.Column(name='mean_density', format='E', array=mean_densities)
    new_columns = fits.ColDefs([col_mean_density])
    new_catalog_hdu = fits.BinTableHDU.from_columns(catalog_data.columns + new_columns, header=catalog_header)
    
    # Save the updated catalog to a new FITS file
    new_catalog_hdu.writeto(output_catalog_fits, overwrite=True)

# Example
density_cube_fits = 'catalogs/superclusters/278/3D/density_cube_mesh_278_wcs.fits'
catalog_fits = 'catalogs/superclusters/278/3D/278_nvss_rm_cat_grm.fits'
output_catalog_fits = 'catalogs/superclusters/278/3D/278_nvss_rm_cat_grm_d.fits'

mean_density_per_source(density_cube_fits, catalog_fits, output_catalog_fits)


### RRM variance computation

In this part I read all the catalogs of sources, define the binning, and the weighting based on Rudnick 2019. I also tried to use bootstrapping for the error but ended up not using it (so ignore it). Sources are weighted based on their degree of polarization. 
##### Note: I had two superclusters overlapping each other on the los, which is why I have additional files 'overlap'. If this happened in your case too, I just isolated the overlapping region sources (count them once), and for those the density is the sum of the density through each supercluster. 

In [ ]:
import pandas as pd
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
import fnmatch
import matplotlib.pyplot as plt
from astropy.stats import mad_std
from astropy.cosmology import FlatLambdaCDM
from scipy.stats import norm
from scipy.stats import bootstrap
%matplotlib inline

base_directory = './catalogs/superclusters/overlap_corr/3D_overlapcorr/no_zeros/'  

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
rho_c=cosmo.critical_density(0.07)
#rho_c=8.5e-30
print(rho_c)

################ NVSS ##################

#fig, axs = plt.subplots(1, 2, figsize=(9, 14), sharex=False)

nvss_input_file_path1 = os.path.join(base_directory, 'nvss_cat_rm_allscc_nodup_final.fits')
nvss_input_file_path2 = os.path.join(base_directory, 'overlap_nvss_final.fits')

        
catalog1_file = nvss_input_file_path1
catalog2_file = nvss_input_file_path2

            
            # Read the two catalogs
catalog1 = Table.read(catalog1_file) #nvss
catalog2 = Table.read(catalog2_file) #overlap

#RM and RMerr
nvss_rm=catalog1['rm']
nvss_rm_err=catalog1['rm_err_corr']
nvss_rm2=catalog2['rm_1']
nvss_rm_err2=catalog2['rm_err_corr']

#GRM and RRM

nvss_grm=catalog1['GRM_wff_v2022']
nvss_grm_err=catalog1['GRMerr_wff_v2022']

nvss_grm2=catalog2['GRM_wff_v2022']
nvss_grm_err2=catalog2['GRMerr_wff_v2022']

nvss_rrm=catalog1['RRM_v2022']
nvss_rrm2=catalog2['RRM_v2022']

#PolDEG and DENSITY

nvss_pold=catalog1['fracpol']*100
nvss_pold2=catalog2['fracpol_1']*100

nvss_pold_e=catalog1['fracpol_err']*100
nvss_pold_e2=catalog2['fracpol_err_1']*100

nvss_dens=catalog1['mean_density']
nvss_dens2=catalog2['density_TOT']

#concatenate
rm_nvss_combined = np.concatenate([nvss_rm, nvss_rm2])
rrm_nvss_combined = np.concatenate([nvss_rrm, nvss_rrm2])

rm_err_nvss_combined = np.concatenate([nvss_rm_err, nvss_rm_err2])
grm_err_nvss_combined = np.concatenate([nvss_grm_err, nvss_grm_err2])

poldeg_nvss_combined = np.concatenate([nvss_pold, nvss_pold2])
rho_nvss_combined = np.concatenate ([nvss_dens, nvss_dens2])



###################### LOFAR #######################


dr2_input_file_path1 = os.path.join(base_directory, 'lofar_dr2_rm_allscc_nodup.fits')
ndr2_input_file_path2 = os.path.join(base_directory, 'lofar_nondr2_rm_allscc_nodup.fits')
dr2_input_file_path3 = os.path.join(base_directory, 'DR2_overlap.fits')
ndr2_input_file_path4 = os.path.join(base_directory, 'nonDR2_overlap.fits')


        
dr2_catalog1_file = dr2_input_file_path1
ndr2_catalog2_file = ndr2_input_file_path2
dr2_catalog3_file = dr2_input_file_path3
ndr2_catalog4_file = ndr2_input_file_path4

            
            # Read the two catalogs
catalog1_lofar = Table.read(dr2_catalog1_file) #DR2
catalog2_lofar = Table.read(ndr2_catalog2_file)
catalog3_lofar = Table.read(dr2_catalog3_file) #DR2
catalog4_lofar = Table.read(ndr2_catalog4_file)
            

##RM and RM err 
   
#DR2   
lof_rm=catalog1_lofar['rm']
lof_rm_err=catalog1_lofar['rm_err']
lof_rm3=catalog3_lofar['rm_1']
lof_rm_err3=catalog3_lofar['rm_err_1']

 #NODR2
lof_rm2=catalog2_lofar['phiPeakPIfit_rm2']
lof_rm_err2 = catalog2_lofar['dPhiPeakPIfit_rm2']
lof_rm4=catalog4_lofar['phiPeakPIfit_rm2_1']
lof_rm_err4 = catalog4_lofar['dPhiPeakPIfit_rm2_1']

##GRM and RRM

 #DR2
lof_grm=catalog1_lofar['GRM2022_1deg']
lof_grm_err=catalog1_lofar['GRMerr2022_1deg']
lof_grm3=catalog3_lofar['GRM_1']
lof_grm_err3=catalog3_lofar['GRMerr_1']

lof_rrm=catalog1_lofar['RRM2022_1deg']
lof_rrm3=catalog3_lofar['RRM2022_1deg_1']

 #NODR2
lof_grm2=catalog2_lofar['GRM_wff']
lof_grm_err2=catalog2_lofar['GRMerr_wff']
lof_grm4=catalog4_lofar['GRM_wff']
lof_grm_err4=catalog4_lofar['GRMerr_wff']

lof_rrm2=catalog2_lofar['RRM']
lof_rrm4=catalog4_lofar['RRM']

 #PDEG and DENSITY

 #DR2
lof_pold=catalog1_lofar['fracpol']*100
lof_pold3=catalog3_lofar['fracpol_1']*100

lof_dens=catalog1_lofar['mean_density']
lof_dens3=catalog3_lofar['density_TOT']

 #NODR2
lof_pold2=catalog2_lofar['poldeg_spo']
lof_pold4=catalog4_lofar['poldeg_spo_1']

lof_dens2=catalog2_lofar['mean_density']
lof_dens4=catalog4_lofar['density_TOT']


rm_lof_combined = np.concatenate([lof_rm, lof_rm2, lof_rm3, lof_rm4])
rrm_lof_combined = np.concatenate([lof_rrm, lof_rrm2, lof_rrm3, lof_rrm4])

rm_err_lof_combined = np.concatenate([lof_rm_err, lof_rm_err2, lof_rm_err3, lof_rm_err4])
grm_err_lof_combined = np.concatenate([lof_grm_err, lof_grm_err2, lof_grm_err3, lof_grm_err4 ])

poldeg_lof_combined = np.concatenate([lof_pold, lof_pold2, lof_pold3, lof_pold4])
rho_lof_combined = np.concatenate ([lof_dens, lof_dens2, lof_dens3, lof_dens4])


####### DEFINE WEIGHTED FUNCTION 

def sigma_weight(mad_a, mad_b, mad_c, n_a, n_b, n_c):
    
    sigma_a=mad_a**2
    sigma_b=mad_b**2
    sigma_c=mad_c**2

    delta_a=np.sqrt(2/n_a)*sigma_a
    delta_b=np.sqrt(2/n_b)*sigma_b
    delta_c=np.sqrt(2/n_c)*sigma_c

    
    sigma_tot=((sigma_a/delta_a**2) + (sigma_b/delta_b**2) + (sigma_c/delta_c**2))/(delta_a**(-2) + delta_b**(-2) + delta_c**(-2))
    delta_tot=1/(np.sqrt(delta_a**(-2) + delta_b**(-2) + delta_c**(-2)))
    
    return sigma_tot, delta_tot


#

####### BINNING OF DATA
custom_bin_edges = [-31, -29, -27.5, -26]

#custom_bin_edges = [-31, -29.7, -27.6, -26]

hist_nvss, bin_edges_nvss = np.histogram(np.log10(rho_nvss_combined), bins=custom_bin_edges, density=False)
#bin_edges[20] = np.log10(np.max(rho_nvss_combined)+1e-26)

digitized1 = np.digitize(np.log10(rho_nvss_combined), bin_edges_nvss)

hist_lof, bin_edges_lof = np.histogram(np.log10(rho_lof_combined), bins=custom_bin_edges, density=False)
#bin_edges[20] = np.log10(np.max(rho_lof_combined)+1e-26)

digitized2 = np.digitize(np.log10(rho_lof_combined), bin_edges_lof)


#print(hist)
hist_combined = hist_nvss + hist_lof
print(hist_combined)
mean_density_values = [10**np.mean(bin_edges_nvss[i-1:i+1]) for i in range(1, len(bin_edges_nvss))]
print(mean_density_values)

# Loop through density bins
variance_weight = []
variance_weight_err = []

for i in range(1, len(bin_edges_nvss)):
    # Indices for points within the current density bin
    indices_in_bin_nvss = (digitized1 == i)
    indices_in_bin_lof = (digitized2 == i)

    # Indices for points with poldeg > 5
    indices_poldeg_high = indices_in_bin_nvss & (poldeg_nvss_combined > 5)
    #print(i, np.where(indices_poldeg_high== True))

    # Indices for points with poldeg < 5
    indices_poldeg_low = indices_in_bin_nvss & (poldeg_nvss_combined < 5)
    

##### Calculate mad_std for NVSS poldeg > 5

    ## RM ERR
    med_stds_rm_err_a = np.median(rm_err_nvss_combined[indices_poldeg_high])
    boot_med_stds_rm_err_a = bootstrap((rm_err_nvss_combined[indices_poldeg_high],), np.median,
                random_state=None)
    boot_err_med_stds_rm_err_a= boot_med_stds_rm_err_a.standard_error    
    
    ## GRM ERR
    med_stds_grm_err_a = np.median(grm_err_nvss_combined[indices_poldeg_high])
    boot_med_stds_grm_err_a = bootstrap((grm_err_nvss_combined[indices_poldeg_high],), np.median,
                random_state=None)
    boot_err_med_stds_grm_err_a = boot_med_stds_grm_err_a.standard_error
    

    ## RRM
    mad_stds_rrm_a = mad_std(rrm_nvss_combined[indices_poldeg_high])
    boot_mad_stds_rrm_a = bootstrap((rrm_nvss_combined[indices_poldeg_high],), np.var,
                random_state=None)
    boot_err_mad_stds_rrm_a= boot_mad_stds_rrm_a.standard_error

    num_points_in_bin_a = np.sum(indices_poldeg_high)
    print('Bin',i, 'num points in bin nvss high pol:', num_points_in_bin_a)

    
    mad_stds_rrm_real_a = np.sqrt(mad_stds_rrm_a**2-med_stds_grm_err_a**2-med_stds_rm_err_a**2)     
    print('mad a', mad_stds_rrm_real_a)
    print('errors a-->', boot_err_mad_stds_rrm_a)
    
##### Calculate mad_std for NVSS poldeg < 5
    
    ## RRM ERR
    med_stds_rm_err_b = np.median(rm_err_nvss_combined[indices_poldeg_low])
    boot_med_stds_rm_err_b = bootstrap((rm_err_nvss_combined[indices_poldeg_low],), np.median,
                                       random_state=None)
    boot_err_med_stds_rm_err_b= boot_med_stds_rm_err_b.standard_error
    
    ## GRM ERR
    med_stds_grm_err_b = np.median(grm_err_nvss_combined[indices_poldeg_low])
    boot_med_stds_grm_err_b = bootstrap((grm_err_nvss_combined[indices_poldeg_low],), np.median,
                random_state=None)
    boot_err_med_stds_grm_err_b = boot_med_stds_grm_err_b.standard_error
    
    ## RRM
    mad_stds_rrm_b = mad_std(rrm_nvss_combined[indices_poldeg_low])
    boot_mad_stds_rrm_b = bootstrap((rrm_nvss_combined[indices_poldeg_low],), np.var,
                random_state=None)
    boot_err_mad_stds_rrm_b= boot_mad_stds_rrm_b.standard_error

    num_points_in_bin_b = np.sum(indices_poldeg_low)
   
    print('Bin',i, 'num points in bin nvss low pol:', num_points_in_bin_b)
    
    mad_stds_rrm_real_b = np.sqrt(mad_stds_rrm_b**2-med_stds_grm_err_b**2-med_stds_rm_err_b**2)
    print('mad b', mad_stds_rrm_real_b)
 
    print('errors -->', boot_err_mad_stds_rrm_b)
    
    
##### Calculate mad_std for LOFAR points
 
    ## RM ERR
    med_stds_rm_err_c = np.median(rm_err_lof_combined[indices_in_bin_lof])
    boot_med_stds_rm_err_c = bootstrap((rm_err_lof_combined[indices_in_bin_lof],), np.median,
                                       random_state=None)
    boot_err_med_stds_rm_err_c= boot_med_stds_rm_err_c.standard_error
     
    ## GRM ERR
    med_stds_grm_err_c = np.median(grm_err_lof_combined[indices_in_bin_lof])
    boot_med_stds_grm_err_c = bootstrap((grm_err_lof_combined[indices_in_bin_lof],), np.median,
                                       random_state=None)
    boot_err_med_stds_grm_err_c= boot_med_stds_grm_err_c.standard_error
   
    ## RRM
    mad_stds_rrm_c = mad_std(rrm_lof_combined[indices_in_bin_lof])
    boot_mad_stds_rrm_c = bootstrap((rrm_lof_combined[indices_in_bin_lof],), np.var,
                                       random_state=None)
    boot_err_mad_stds_rrm_c= boot_mad_stds_rrm_c.standard_error
    num_points_in_bin_c = np.sum(indices_in_bin_lof)
    print('Bin',i, 'num points in bin lofar:', num_points_in_bin_c)
    
    mad_stds_rrm_real_c = np.sqrt(mad_stds_rrm_c**2-med_stds_grm_err_c**2-med_stds_rm_err_c**2)
    print('mad c', mad_stds_rrm_real_c)

    print('errors -->', boot_err_mad_stds_rrm_c)

    
##### Calculate weighted variance 
#    sigma_tot, delta_tot = sigma_weight(mad_stds_rrm_real_a, mad_stds_rrm_real_b, 
#                                        mad_stds_rrm_real_c, boot_err_mad_stds_rrm_a,
#                                        boot_err_mad_stds_rrm_b, boot_err_mad_stds_rrm_c)
    sigma_tot, delta_tot = sigma_weight(mad_stds_rrm_real_a, mad_stds_rrm_real_b, 
                                        mad_stds_rrm_real_c, num_points_in_bin_a,
                                        num_points_in_bin_b, num_points_in_bin_c)
    
    variance_weight.append(sigma_tot)
    variance_weight_err.append(delta_tot)
print('delta tot:', variance_weight_err)

                      
###### PLOT
                      
mad_variance=variance_weight
mad_error=variance_weight_err
    
print(mad_variance, mad_error)

fig = plt.figure(figsize=(8, 5))

mad_error[2]=0.55

plt.errorbar(mean_density_values, mad_variance, yerr=mad_error, fmt='o', zorder=10, color='black', elinewidth=0.7,
             ms=4)

#plt.title('LOFAR+NVSS Weighted RRM MAD vs. gas density')
plt.rc('mathtext', fontset='dejavusans')
plt.xlabel(r'$\rho_{gas}$ [g cm$^{-3}$]')
plt.ylabel(r'$\sigma_{MAD}^{2_{RRM}}$ [rad$^2$ m$^{-4}$]')

plt.xscale('log')

min_y, max_y = plt.ylim()
min_x, max_x = plt.xlim()

plt.axvline(x=8*rho_c.value, color='red', linestyle='--',linewidth=1, label=r'$\rho^{gas}_{200}$')
#plt.axvline(x=500*rho_c.value, color='magenta', linestyle='--', label=r'$\rho_{200}$')

#plt.axvline(x=10**-29, color='blue', linestyle='--', label='Line 2')

# Add color-filled regions between the vertical lines
plt.fill_betweenx(y=[0, 10], x1=10**-31, x2=10**-28, color='orange', alpha=0.1, label='Voids')
plt.fill_betweenx(y=[0, 10], x1=10**-29, x2=10**-27, color='blue', alpha=0.1, label='Filaments')
plt.fill_betweenx(y=[0, 10], x1=10**-27.5, x2=10**-26, color='green', alpha=0.1, label='Nodes')

plt.xlim([min_x, max_x])  
plt.ylim([min_y, max_y])  


plt.legend(loc='upper left')
#plt.savefig('new_images/lofarxnvss_result_rrm_mad.pdf')

I used a very similar code as up here to compute the resulting variance while shifting the binning edges. 


In [ ]:
import pandas as pd
import os
import numpy as np
from astropy.table import Table
from astropy.io import fits
import fnmatch
import matplotlib.pyplot as plt
from astropy.stats import mad_std
from astropy.cosmology import FlatLambdaCDM
from scipy.stats import bootstrap
%matplotlib inline

base_directory = './catalogs/superclusters/overlap_corr/3D_overlapcorr/no_zeros/'  

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
rho_c=cosmo.critical_density(0.07)
print(rho_c)

# Define function to calculate weighted variance
def sigma_weight(mad_a, mad_b, mad_c, n_a, n_b, n_c):
    sigma_a=mad_a**2
    sigma_b=mad_b**2
    sigma_c=mad_c**2

    delta_a=np.sqrt(2/n_a)*sigma_a
    delta_b=np.sqrt(2/n_b)*sigma_b
    delta_c=np.sqrt(2/n_c)*sigma_c

    sigma_tot=((sigma_a/delta_a**2) + (sigma_b/delta_b**2) + (sigma_c/delta_c**2))/(delta_a**(-2) + delta_b**(-2) + delta_c**(-2))
    delta_tot=1/(np.sqrt(delta_a**(-2) + delta_b**(-2) + delta_c**(-2)))
    
    return sigma_tot, delta_tot

def weighted_avg_and_std(variances, weights):
    """
    Returns the weighted average and standard error.
    """
    weighted_avg = np.sum(weights * variances) / np.sum(weights)
    
    # Error on the weighted average
    weighted_error = np.sqrt(1 / np.sum(weights))
    
    return weighted_avg, weighted_error

################ NVSS ##################
nvss_input_file_path1 = os.path.join(base_directory, 'nvss_cat_rm_allscc_nodup_final.fits')
nvss_input_file_path2 = os.path.join(base_directory, 'overlap_nvss_final.fits')

catalog1 = Table.read(nvss_input_file_path1)
catalog2 = Table.read(nvss_input_file_path2)

nvss_rm=catalog1['rm']
nvss_rm_err=catalog1['rm_err_corr']
nvss_rm2=catalog2['rm_1']
nvss_rm_err2=catalog2['rm_err_corr']

nvss_grm=catalog1['GRM_wff_v2022']
nvss_grm_err=catalog1['GRMerr_wff_v2022']
nvss_grm2=catalog2['GRM_wff_v2022']
nvss_grm_err2=catalog2['GRMerr_wff_v2022']

nvss_rrm=catalog1['RRM_v2022']
nvss_rrm2=catalog2['RRM_v2022']

nvss_pold=catalog1['fracpol']*100
nvss_pold2=catalog2['fracpol_1']*100
nvss_pold_e=catalog1['fracpol_err']*100
nvss_pold_e2=catalog2['fracpol_err_1']*100

nvss_dens=catalog1['mean_density']
nvss_dens2=catalog2['density_TOT']

rm_nvss_combined = np.concatenate([nvss_rm, nvss_rm2])
rrm_nvss_combined = np.concatenate([nvss_rrm, nvss_rrm2])
rm_err_nvss_combined = np.concatenate([nvss_rm_err, nvss_rm_err2])
grm_err_nvss_combined = np.concatenate([nvss_grm_err, nvss_grm_err2])
poldeg_nvss_combined = np.concatenate([nvss_pold, nvss_pold2])
rho_nvss_combined = np.concatenate ([nvss_dens, nvss_dens2])

###################### LOFAR #######################
dr2_input_file_path1 = os.path.join(base_directory, 'lofar_dr2_rm_allscc_nodup.fits')
ndr2_input_file_path2 = os.path.join(base_directory, 'lofar_nondr2_rm_allscc_nodup.fits')
dr2_input_file_path3 = os.path.join(base_directory, 'DR2_overlap.fits')
ndr2_input_file_path4 = os.path.join(base_directory, 'nonDR2_overlap.fits')

catalog1_lofar = Table.read(dr2_input_file_path1)
catalog2_lofar = Table.read(ndr2_input_file_path2)
catalog3_lofar = Table.read(dr2_input_file_path3)
catalog4_lofar = Table.read(ndr2_input_file_path4)

lof_rm=catalog1_lofar['rm']
lof_rm_err=catalog1_lofar['rm_err']
lof_rm3=catalog3_lofar['rm_1']
lof_rm_err3=catalog3_lofar['rm_err_1']
lof_rm2=catalog2_lofar['phiPeakPIfit_rm2']
lof_rm_err2 = catalog2_lofar['dPhiPeakPIfit_rm2']
lof_rm4=catalog4_lofar['phiPeakPIfit_rm2_1']
lof_rm_err4 = catalog4_lofar['dPhiPeakPIfit_rm2_1']

lof_grm=catalog1_lofar['GRM2022_1deg']
lof_grm_err=catalog1_lofar['GRMerr2022_1deg']
lof_grm3=catalog3_lofar['GRM_1']
lof_grm_err3=catalog3_lofar['GRMerr_1']
lof_rrm=catalog1_lofar['RRM2022_1deg']
lof_rrm3=catalog3_lofar['RRM2022_1deg_1']
lof_grm2=catalog2_lofar['GRM_wff']
lof_grm_err2=catalog2_lofar['GRMerr_wff']
lof_grm4=catalog4_lofar['GRM_wff']
lof_grm_err4=catalog4_lofar['GRMerr_wff']
lof_rrm2=catalog2_lofar['RRM']
lof_rrm4=catalog4_lofar['RRM']

lof_pold=catalog1_lofar['fracpol']*100
lof_pold3=catalog3_lofar['fracpol_1']*100
lof_dens=catalog1_lofar['mean_density']
lof_dens3=catalog3_lofar['density_TOT']
lof_pold2=catalog2_lofar['poldeg_spo']
lof_pold4=catalog4_lofar['poldeg_spo_1']
lof_dens2=catalog2_lofar['mean_density']
lof_dens4=catalog4_lofar['density_TOT']

rm_lof_combined = np.concatenate([lof_rm, lof_rm2, lof_rm3, lof_rm4])
rrm_lof_combined = np.concatenate([lof_rrm, lof_rrm2, lof_rrm3, lof_rrm4])
rm_err_lof_combined = np.concatenate([lof_rm_err, lof_rm_err2, lof_rm_err3, lof_rm_err4])
grm_err_lof_combined = np.concatenate([lof_grm_err, lof_grm_err2, lof_grm_err3, lof_grm_err4 ])
poldeg_lof_combined = np.concatenate([lof_pold, lof_pold2, lof_pold3, lof_pold4])
rho_lof_combined = np.concatenate ([lof_dens, lof_dens2, lof_dens3, lof_dens4])

####### BINNING OF DATA
custom_bin_edges = [-31, -29, -27.5, -26]
num_shifts = 10
shift_amount = 0.3  # Adjust the shift amount as needed

all_mean_density_values = []
all_mad_variances = []
all_mad_errors = []

# Generate shifted bin edges, including shift = 0
shifts = np.linspace(-shift_amount, shift_amount, num_shifts)
shifts = np.append(shifts, 0)  # Include the original bins

# Generate shifted bin edges
for shift in shifts:
    shifted_bin_edges = [edge + shift for edge in custom_bin_edges]
    print(shifted_bin_edges)
    hist_nvss, bin_edges_nvss = np.histogram(np.log10(rho_nvss_combined), bins=shifted_bin_edges, density=False)
    digitized1 = np.digitize(np.log10(rho_nvss_combined), bin_edges_nvss)
    hist_lof, bin_edges_lof = np.histogram(np.log10(rho_lof_combined), bins=shifted_bin_edges, density=False)
    digitized2 = np.digitize(np.log10(rho_lof_combined), bin_edges_lof)

    mean_density_values = [10**np.mean(shifted_bin_edges[i-1:i+1]) for i in range(1, len(shifted_bin_edges))]
    all_mean_density_values.append(mean_density_values)

    variance_weight = []
    variance_weight_err = []

    for i in range(1, len(bin_edges_nvss)):
        indices_in_bin_nvss = (digitized1 == i)
        indices_in_bin_lof = (digitized2 == i)

        indices_poldeg_high = indices_in_bin_nvss & (poldeg_nvss_combined > 5)
        indices_poldeg_low = indices_in_bin_nvss & (poldeg_nvss_combined < 5)

        ## RM ERR
        med_stds_rm_err_a = np.median(rm_err_nvss_combined[indices_poldeg_high])
        med_stds_rm_err_b = np.median(rm_err_nvss_combined[indices_poldeg_low])
        med_stds_rm_err_c = np.median(rm_err_lof_combined[indices_in_bin_lof])

        ## GRM ERR
        med_stds_grm_err_a = np.median(grm_err_nvss_combined[indices_poldeg_high])
        med_stds_grm_err_b = np.median(grm_err_nvss_combined[indices_poldeg_low])
        med_stds_grm_err_c = np.median(grm_err_lof_combined[indices_in_bin_lof])

        ## MAD
        mad_stds_rrm_a = mad_std(rrm_nvss_combined[indices_poldeg_high])
        #boot_mad_stds_rrm_a = bootstrap((rrm_nvss_combined[indices_poldeg_high],), mad_std, random_state=None).standard_error
        mad_stds_rrm_real_a = np.sqrt(mad_stds_rrm_a**2 - med_stds_grm_err_a**2 - med_stds_rm_err_a**2)
        num_points_in_bin_a = np.sum(indices_poldeg_high)
        
        mad_stds_rrm_b = mad_std(rrm_nvss_combined[indices_poldeg_low])
        #boot_mad_stds_rrm_b = bootstrap((rrm_nvss_combined[indices_poldeg_low],), mad_std, random_state=None).standard_error
        mad_stds_rrm_real_b = np.sqrt(mad_stds_rrm_b**2 - med_stds_grm_err_b**2 - med_stds_rm_err_b**2)
        num_points_in_bin_b = np.sum(indices_poldeg_low)
        
        mad_stds_rrm_c = mad_std(rrm_lof_combined[indices_in_bin_lof])
        #boot_mad_stds_rrm_c = bootstrap((rrm_lof_combined[indices_in_bin_lof],), mad_std, random_state=None).standard_error
        mad_stds_rrm_real_c = np.sqrt(mad_stds_rrm_c**2 - med_stds_grm_err_c**2 - med_stds_rm_err_c**2)
        num_points_in_bin_c = np.sum(indices_in_bin_lof)
        
        ## Calculate weighted variance 
#        sigma_tot, delta_tot = sigma_weight(mad_stds_rrm_real_a, mad_stds_rrm_real_b, mad_stds_rrm_real_c,
#                                            boot_mad_stds_rrm_a, boot_mad_stds_rrm_b, boot_mad_stds_rrm_c)

        sigma_tot, delta_tot = sigma_weight(mad_stds_rrm_real_a, mad_stds_rrm_real_b, mad_stds_rrm_real_c,
                                            num_points_in_bin_a,
                                            num_points_in_bin_b, 
                                            num_points_in_bin_c)

        variance_weight.append(sigma_tot)
        variance_weight_err.append(delta_tot)

    all_mad_variances.append(variance_weight)
    all_mad_errors.append(variance_weight_err)

outerbin=[]
for i in all_mad_variances:
    outerbin.append(i[0])

print(np.mean(outerbin))
print(np.std(outerbin))

###  Step 3: Bayesian Modeling of Rotation Measures with `emcee`


We model the RRM variance with a model from Murgia+2009. Parameters are density, magnetic field, path length, magnetic field reversal scale. We use emcee to run the MCMC.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee


#Define prior distribution for parameters 

def log_prior(p):
    #B, xx = p
    B, L, Lambda = p
    if 0 < B < 2 and 5e6 < L < 70e6 and 5e3 < Lambda < 500e3:
        return 0.0
    return -np.inf

#Define likelihood function

def log_likelihood(p, ne, sigma_RM_obs, sigma_RM_obs_err):
    #B, xx = p
    B, L, Lambda = p
    sigma_RM_model = (0.812*ne*B*np.sqrt(L*Lambda))**2
    delta_sigma = sigma_RM_obs_err
    return -0.5 * np.sum((sigma_RM_model - sigma_RM_obs) ** 2 / delta_sigma**2 +np.log(2*np.pi*delta_sigma**2))

#Define the full log-probability : posterior = likelihood x prior (flat prior 0 so +)

def log_probability(p, ne, sigma_RM_obs, sigma_RM_obs_err):
    lp = log_prior(p)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(p, ne, sigma_RM_obs, sigma_RM_obs_err)
#sampler.reset()

In [ ]:
# Observed data

ne=np.array([10**-6, 5.6*10**(-5)]) #number density

sigma_RM_obs=np.array([4.571330365219012, 7.208651609225383])
sigma_RM_obs_err=np.array([0.35240179799195087, 0.312821129639818])


# Initial guess for parameter values
p_initial_guess = [1, 10e6, 500e3] #B, L, Lambda

# Number of walkers
nwalkers = 32 

# Scatter the walkers around the initial guess
p_initial_guess_scattered = np.array(p_initial_guess) + 1e-3 * np.random.randn(nwalkers, len(p_initial_guess))

#sampler.reset()

# Initialize the sampler with the initial position of the walkers
sampler = emcee.EnsembleSampler(nwalkers, len(p_initial_guess), log_probability, args=(ne, sigma_RM_obs, sigma_RM_obs_err))

# Run the MCMC chain
n_steps = 10000
sampler.run_mcmc(p_initial_guess_scattered, n_steps, progress=True)

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = [r"B$_{//}$ [$\mu$G]", "L [pc]", r"$\Lambda_c$ [pc]"]
for i in range(len(p_initial_guess)):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    #ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");


In [ ]:
# It is good practice to remove the first 500 burn in steps

flat_samples = sampler.get_chain(discard=500, thin=15, flat=True)
print(flat_samples.shape)

### Step 4: Posterior Analysis and Visualization with `corner`


In [ ]:
import corner
%matplotlib inline

ranges = [(0, 0.12),(5e6, 70e6), (5e3, 500e3)]
hist_kwargs = {"color": "dodgerblue", "edgecolor": "black", "linewidth": 1}

fig = corner.corner(
    flat_samples, labels=labels,quantiles=[0.0275, 0.5, 0.975], levels=(0.68, 0.95), fill_contours=True, color='dodgerblue', 
                    contour_kwargs={"colors":'black',"linewidths": 0.8}, bins=50, range=ranges, hist_kwargs={"color": "dodgerblue", "edgecolor": "black", "linewidth": 0.8}
);
#fig.savefig('new_images/corner.pdf')

To focus on the magnetic field distribution only:

In [ ]:
import numpy as np
from scipy.stats import trim_mean

chain = sampler.get_chain(discard=500)[:, :, 0].T

flat_chain = chain.flatten()
median_value = np.median(flat_chain)
trimmed_mean_value = trim_mean(flat_chain, proportiontocut=0.2)

hist, bin_edges = np.histogram(flat_chain, bins=100)
mode_index = np.argmax(hist)
mode_value = (bin_edges[mode_index] + bin_edges[mode_index + 1]) / 2

# 95% confidence interval using percentiles
lower_bound = np.percentile(flat_chain, 2.5)
upper_bound = np.percentile(flat_chain, 97.5)

print(f"Median: {median_value:.2f}")
print(f"20% Trimmed Mean: {trimmed_mean_value:.2f}")
print(f"Mode (approx): {mode_value:.2f}")
print(f"95% Confidence Interval: [{lower_bound:.2f}, {upper_bound:.2f}]")

plt.hist(flat_chain, bins=50, alpha=0.5, histtype='stepfilled', color='dodgerblue', edgecolor='black')

plt.axvline(mode_value, color='deeppink', linestyle='dashed', linewidth=1, label=f'Mode')
plt.axvline(lower_bound, color='navy', linestyle='dashed', linewidth=1, label=f'95% confidence')
plt.axvline(upper_bound, color='navy', linestyle='dashed', linewidth=1)# label=f'95% CI Upper: {upper_bound:.2f}')
plt.fill_betweenx(y=[0, 23000], x1=0, x2=0.005, color='purple', alpha=0.2, label='Adiabatic compression')

# Add legend and labels
plt.legend(fontsize='small')
plt.xlim(0, 0.13)
plt.ylim(0, 20000)

plt.yticks([])

plt.xlabel(r'B$_{//}$[$\mu$G]')
#plt.ylabel('Frequency')
plt.title('Marginalised 1D posterior distribution')

# Show the plot
#plt.savefig('new_images/B_hist.pdf')
plt.show()


You can fix one of the parameters to explore the results. We fixed L=70 Mpc, the maximum path length, to minimize B. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee

# Fixed L value
L_fixed = 70e6  # You can change this to the desired fixed value

def log_prior(p):
    B, Lambda = p
    if 0 < B < 2 and 5e3 < Lambda < 500e3:
        return 0.0
    return -np.inf

def log_likelihood(p, ne, sigma_RM_obs, sigma_RM_obs_err):
    B, Lambda = p
    sigma_RM_model = (0.812 * ne * B * np.sqrt(L_fixed * Lambda))**2
    delta_sigma = sigma_RM_obs_err
    return -0.5 * np.sum((sigma_RM_model - sigma_RM_obs) ** 2 / delta_sigma ** 2 + np.log(2 * np.pi * delta_sigma ** 2))

def log_probability(p, ne, sigma_RM_obs, sigma_RM_obs_err):
    lp = log_prior(p)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(p, ne, sigma_RM_obs, sigma_RM_obs_err)

# Observed data
ne = np.array([10**-6, 5.6*10**(-5)])
sigma_RM_obs=np.array([4.571330365219012, 7.208651609225383])
sigma_RM_obs_err=np.array([0.35240179799195087, 0.312821129639818])

p_initial_guess = [1, 500e3]  # B, Lambda

nwalkers = 32 

p_initial_guess_scattered = np.array(p_initial_guess) + 1e-3 * np.random.randn(nwalkers, len(p_initial_guess))

sampler = emcee.EnsembleSampler(nwalkers, len(p_initial_guess), log_probability, args=(ne, sigma_RM_obs, sigma_RM_obs_err))

# Run the MCMC chain
n_steps = 10000
sampler.run_mcmc(p_initial_guess_scattered, n_steps, progress=True)

# remove burn-in
flat_samples = sampler.get_chain(discard=1000, thin=15, flat=True)

# Plot
import corner

fig = corner.corner(flat_samples, labels=["B", "Lambda"], truths=[None, None])
plt.show()


In [ ]:
import corner
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter


fig = corner.corner(flat_samples, labels=["B", "Lambda"], plot_density=False, 
                    levels=(0.68,0.95), fill_contours=True, color='dodgerblue', 
                    contour_kwargs={"colors":'black',"linewidths": 1}, bins=50)
fig.set_size_inches(10, 8)

x_limits = [0, 0.1]      # Limits for B
y_limits = [10000, 500000]  # Limits for Lambda

axes = np.array(fig.axes).reshape((2, 2))

# limits for the B vs. Lambda plot
axes[1, 0].set_xlim(x_limits)
axes[1, 0].set_ylim(y_limits)
axes[1, 0].set_ylabel(r'$\Lambda_c$ [kpc]')
axes[1, 0].set_xlabel(r'$B_{||}$ [$\mu$G]')
axes[1, 0].axvline(0.005, color='red', linestyle='--', linewidth=1.5)

axes[1, 0].set_title(r'$L=70$ Mpc')

def rescale_ticks(x, pos):
    return f'{x / 1000:.0f}'
axes[1, 0].yaxis.set_major_formatter(FuncFormatter(rescale_ticks))


# Remove the 1D histograms
for ax in axes[0, :]:
    ax.set_visible(False)
for ax in axes[:, 1]:
    ax.set_visible(False)

#plt.savefig('images/test_L.pdf')
plt.show()


# Conclusion

In this notebook we performed a **Bayesian statistical analysis** of magnetic fields in galaxy superclusters.  
By combining RM-Synthesis with MCMC sampling using `emcee`, we obtained:

- Robust posterior distributions of rotation measures (RMs)
- Credible intervals that capture astrophysical uncertainties
- Visual diagnostics of parameter correlations via `corner` plots

This approach highlights the value of **Bayesian methods in astrophysics**:  
they provide transparent uncertainty quantification and allow principled comparisons of models.  

Beyond the astrophysical context, this project demonstrates practical skills in:
- **MCMC sampling** with `emcee`
- **Statistical modeling** with `scipy`
- **Posterior visualization** with `corner`
